In [9]:
import os

import pandas as pd
import numpy as np
import openpyxl as ox
import re

from openpyxl import load_workbook
from openpyxl import workbook
from openpyxl.styles import Font
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.cell.cell import WriteOnlyCell

from datetime import datetime, timedelta

import warnings
warnings.filterwarnings("ignore")

import string

import pyodbc

data = pd.read_excel(r'C:\Users\ADavydovskiy\Desktop\Тест.xlsx')

In [10]:
conn = pyodbc.connect(r'Driver={SQL Server};Server=MSK1-BIDB01;Database=DWH;Trusted_Connection=yes;')
cursor = conn.cursor()

Employees = f'''

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED

SELECT [EMPLOYEES]
      ,[LOG]
      ,[SP]

      ,CASE

       WHEN KC like '%4%' THEN 'КЦ 4'
       WHEN KC like '%3%' THEN 'КЦ 3'
       WHEN KC like '%центр 3%' THEN 'КЦ 3'
       WHEN KC like '%продаж 18%' THEN 'ОП 18'
             
       ELSE KC
  END
  as "KC"

         ,CASE

       WHEN [KC] like '%4%' THEN SUBSTRING([ОП],CHARINDEX(' ',[ОП])+1,LEN([ОП])) + ' ВР'
       WHEN ([ОП] in ('ОП 9')) and ([GP] in ('.1')) THEN [ОП] + [GP]
       WHEN ([ОП] in ('ОП 2')) and ([GP] in ('.1')) THEN 'ОП 8'
       WHEN ([ОП] in ('ОП 10')) and ([GP] in ('.2')) THEN 'ОП 10.2'
       WHEN ([ОП] in ('ОП 5')) and ([GP] in ('.1')) THEN [ОП] + [GP]
       WHEN ([ОП] in ('3 ЯР')) and ([GP] in ('.1')) THEN '3.1 ЯР'
       WHEN ([KC] in ('Отдел прямых продаж'))  THEN 'ОПП'
           WHEN [KC] like '%продаж 18%'  THEN 'ОП 18'  
       ELSE [ОП]
  END
  as "ОП"
      
               ,[PHONE]
      ,[STATUS]
      ,[ID_EMPL]
      ,[ID_ORG]

  FROM [DWH].[dbo].[KHTS_EMPL]
  where 
  --KC like '%4%' 
  [SP] in ('КД')


'''

Employees = pd.read_sql_query(Employees, conn)

In [11]:
data = data.drop('ID сделки', axis=1)
data = data.fillna('Пусто')

In [12]:
data = data.loc[data['№ Договора'] != "Пусто"]
data['ID лида'] = data['ID лида'].astype(int)
#data['Дата оплаты по квитанции'] = data['Дата оплаты по квитанции']

In [13]:
l = [f", '{str(el)}'" for el in data['ID лида']]

l = list(l)

l = ''.join(l)

In [14]:
request_string_leads = f'''

SELECT DISTINCT
       D.CODE as "id deal crm"
	  ,R.[CODE] as "id lead crm"  


FROM [DWH].[dbo].DIC_DEAL D
		  LEFT JOIN [DWH].[dbo].[DIC_EMPLOYEES] E ON D.ID_EMPLOYEES = E.ID_EMPLOYEES
		  LEFT JOIN [DWH].[dbo].[ASS_EMPLOYEE_AND_ORGSTRUCTURE] ASS_OS ON E.ID_EMPLOYEES = ASS_OS.ID_EMPLOYEES
		  LEFT JOIN [DWH].[dbo].[v_DIC_ORGSTRUCTURE] org on org.ID_ORGSTRUCTURE = ASS_OS.ID_ORGSTRUCTURE
		 
		  --подсоединение заявок
		  LEFT JOIN [DWH].[dbo].ASS_REQUEST_DEAL ASS_RD ON ASS_RD.ID_DEAL = D.ID_DEAL
		  LEFT JOIN [DWH].[dbo].[DIC_REQUEST] R ON ASS_RD.ID_REQUEST = R.ID_REQUEST

		   -- источник
                
            LEFT JOIN [DWH].[STAGE].[CRM_B_UTS_CRM_LEAD] UL WITH(NOLOCK) ON UL.VALUE_ID = R.CODE
            LEFT JOIN [DWH].[STAGE].[CRM_B_CRM_LEAD] L WITH(NOLOCK) ON L.ID = R.CODE
            INNER JOIN (SELECT [NAME], [STATUS_ID]
		                FROM [DWH].[STAGE].[CRM_B_CRM_STATUS] WITH(NOLOCK)
		                WHERE [ENTITY_ID] = 'SOURCE') SRC ON SRC.[STATUS_ID] = L.[SOURCE_ID]		
       
                    
                    
		    --объявления
		  LEFT JOIN [DWH].[dbo].ASS_AD_REQUEST ASS_AR ON ASS_AR.[ID_TYPE_ASS_AD_REQUEST] = 0   --Связь с учетом всех коректировок
										 AND ASS_AR.ID_REQUEST = R.ID_REQUEST   
		  LEFT JOIN [DWH].[dbo].DIC_AD AD ON AD.ID_AD = ASS_AR.ID_AD
		  LEFT JOIN [DWH].[dbo].DIC_AD_GROUP AG ON AD.ID_AD_GROUP = AG.ID_AD_GROUP
		  LEFT JOIN [DWH].[dbo].DIC_AD_CAMPAIGN AC ON AG.ID_AD_CAMPAIGN = AC.ID_AD_CAMPAIGN
   		  --подсоединение инвойсов к сделке
		  LEFT JOIN [DWH].[dbo].ASS_DEAL_INVOICE ASS_D_I ON ASS_D_I.ID_DEAL = D.ID_DEAL
		  LEFT JOIN [DWH].[dbo].DIC_INVOICE D_I ON ASS_D_I.ID_INVOICE = D_I.ID_INVOICE
		  LEFT JOIN [DWH].[dbo].DIC_STATUS_INVOICE S_I ON S_I.ID_STATUS_INVOICE = D_I.ID_STATUS_INVOICE
		  --каталог продуктов сделки
		  LEFT JOIN [DWH].[dbo].ASS_DEAL_CATALOG_PRODUCT ASS_P ON D.ID_DEAL = ASS_P.ID_DEAL
		  LEFT JOIN [DWH].[dbo].[DIC_CATALOG_PRODUCT] cp on cp.ID_CATALOG_PRODUCT = ASS_P.ID_CATALOG_PRODUCT
		  -- статус сделки
		  left join [DWH].[dbo].[DIC_STATUS_DEAL] sd on sd.ID_STATUS_DEAL = d.ID_STATUS_DEAL
		  --contact
		  left join [DWH].[dbo].[DIC_CLIENT_CRM] cc on cc.[ID_CLIENT_CRM] = d.[ID_CLIENT_CRM]
		  left join [DWH].[dbo].[DIC_CLIENT_CRM_PHONE] PhHome ON PhHome.[ID_CLIENT_CRM]=d.[ID_CLIENT_CRM] AND PhHome.[ID_TYPE_CLIENT_CRM_PHONE]=1	--	Домашний
		  left join [DWH].[dbo].[DIC_CLIENT_CRM_PHONE] PhMob ON PhMob.[ID_CLIENT_CRM]=d.[ID_CLIENT_CRM] AND PhMob.[ID_TYPE_CLIENT_CRM_PHONE]=2		--Мобильный
		  left join [DWH].[dbo].[DIC_CLIENT_CRM_PHONE] PhJob ON PhJob.[ID_CLIENT_CRM]=d.[ID_CLIENT_CRM] AND PhJob.[ID_TYPE_CLIENT_CRM_PHONE]=5		--Рабочий
		  left join [DWH].[dbo].[DIC_CLIENT_CRM_PHONE] PhOst ON PhOst.[ID_CLIENT_CRM]=d.[ID_CLIENT_CRM] AND PhOst.[ID_TYPE_CLIENT_CRM_PHONE]=3		--Прочий
		  -- group product deal
		  left join [DWH].[dbo].[DIC_GROUP_PRODUCT_DEAL] dgp on dgp.[ID_GROUP_PRODUCT_DEAL] = d.[GROUP_PRODUCT_ID]
		  -- расширение свойств сделок
		  left join [DWH].[dbo].[DIC_DEAL_STAT] utsD ON D.[ID_DEAL]=utsD.[ID_DEAL]
		  -- продукт бюджета
			LEFT JOIN [DWH].[dbo].ASS_REQUEST_PRODUCT_BUDGET RPB With(nolock) ON R.ID_REQUEST=RPB.ID_REQUEST and R.DATE_CREATE= RPB.R_DATE
			LEFT JOIN [DWH].[dbo].DIC_PRODUCT_BUDGET PB With(nolock) ON RPB.ID_PRODUCT_BUDGET=PB.ID_PRODUCT_BUDGET
			--дела
			LEFT JOIN [DWH].[stage].[CRM_b_crm_act] CA ON CA.[OWNER_ID] = D.[CODE]
		  
  
WHERE
  D.ID_DEAL <> - 1
  and R.CODE in ('5599690' {l})

'''



In [15]:
leads = pd.read_sql_query(request_string_leads, conn)

In [22]:
MonthDict={ 1 : "Январь",
       2 : "Февраль",
       3 : "Март",
       4 : "Апрель",
       5 : "Май",
       6 : "Июнь",
       7 : "Июль",
       8 : "Август",
       9 : "Сентябрь",
       10 : "Октябрь",
       11 : "Ноябрь",
       12 : "Декабрь"
          }

In [31]:
leads.columns = ('ID сделки', 'ID лида')
data = data.merge(leads, how="left", on="ID лида")


for index in data['Дата оплаты по квитанции']:
    data.loc[data['Дата оплаты по квитанции'] == index, 'месяц оплаты'] = MonthDict[pd.to_datetime(index).month]
    
data.columns

Index(['Unnamed: 0', 'ID встречи', 'ID счета', 'Дата оплаты по квитанции',
       'ID лида', 'Абитуриент', 'Телефон', 'Ответственный',
       'Структурное подразделение', '№ Договора', 'Ставка 1С',
       'Скидка (из товара)', 'Мотиватор входа', 'Мотиватор выхода',
       'Коэффициент 1', 'Коэффициент 2', 'Сумма оплаты Итог', '1ый семестр',
       'Чистая выручка', 'Дата разнесения баланса', 'Дата подачи квитанции',
       'ID сделки_x', 'ID сделки_y', 'ID сделки_x', 'ID сделки_y',
       'ID сделки_x', 'ID сделки_y', 'ID сделки_x', 'ID сделки_y',
       'месяц оплаты', 'ID сделки_x', 'ID сделки_y'],
      dtype='object')

In [32]:
data

,Unnamed: 0,ID встречи,ID счета,Дата оплаты по квитанции,ID лида,Абитуриент,Телефон,Ответственный,Структурное подразделение,№ Договора,...,ID сделки_y,ID сделки_x,ID сделки_y,ID сделки_x,ID сделки_y,ID сделки_x,ID сделки_y,месяц оплаты,ID сделки_x,ID сделки_y
0,137738260.0,977160.0,0.0,26.05.2023,46184797,Черняев Денис Юрьевич,79135968772.0,(IKotliarov) Игорь Котляров,Холдинг «Синергия»,1102447/23,...,5620510,5620510,5620510,5620510,5620510,5620510,5620510,Май,5620510,5620510
1,137736738.0,975590.0,0.0,26.05.2023,46526049,Нигматулин Рамиз Рустамович,79015004701.0,(DBirin) Дмитрий Бирин,Холдинг «Синергия»,1102265/23,...,5640106,5640106,5640106,5640106,5640106,5640106,5640106,Май,5640106,5640106
2,137730341.0,Пусто,0.0,26.05.2023,46440174,Зайцев Павел Андреевич,79265762667.0,(ABushmanov) Алексей Бушманов,Холдинг «Синергия»,1093235/23,...,5623433,5623433,5623433,5623433,5623433,5623433,5623433,Май,5623433,5623433
3,137713462.0,Пусто,0.0,26.05.2023,46771128,Марин Александр Сергеевич,79154207300.0,(MVishniakova) Марина Вишнякова,Холдинг «Синергия»,1102067/23,...,5655801,5655801,5655801,5655801,5655801,5655801,5655801,Май,5655801,5655801
4,137736758.0,975745.0,0.0,26.05.2023,46506291,Ломако Дарья Сергеевна,79623700685.0,(DSakhatov) Данил Сахатов,Холдинг «Синергия»,1099230/23,...,5641152,5641152,5641152,5641152,5641152,5641152,5641152,Май,5641152,5641152
5,137736764.0,963353.0,0.0,26.05.2023,46193870,Смагина Екатерина Иосифовна,89256595835.0,(AAkibaeva) Анита Акибаева,Холдинг «Синергия»,1082440/23,...,5591283,5591283,5591283,5591283,5591283,5591283,5591283,Май,5591283,5591283
6,137736768.0,Пусто,0.0,26.05.2023,45390564,Иванов Артем Владимирович,79177317405.0,(DMakarova) Дарья Макарова,Холдинг «Синергия»,1095985/23,...,5581470,5581470,5581470,5581470,5581470,5581470,5581470,Май,5581470,5581470
7,137736771.0,976370.0,0.0,26.05.2023,46632896,Зотов Илья Михайлович,89693497530.0,(AAkibaeva) Анита Акибаева,Холдинг «Синергия»,1100238/23,...,5644467,5644467,5644467,5644467,5644467,5644467,5644467,Май,5644467,5644467
8,137724754.0,Пусто,0.0,26.05.2023,46011367,Платонов Никита Юрьевич,79227745923.0,(NKubar) Наталья Кубарь,Холдинг «Синергия»,1074191/23,...,5575012,5575012,5575012,5575012,5575012,5575012,5575012,Май,5575012,5575012
9,137724761.0,Пусто,0.0,26.05.2023,46018066,Зайцева Наталья Алексеевна,79852611238.0,(VZhukov) Вадим Жуков,Холдинг «Синергия»,1074478/23,...,5575619,5575619,5575619,5575619,5575619,5575619,5575619,Май,5575619,5575619
